In [1]:
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor


# Start the timer
start_time = time.time()

base_url = 'https://www.subito.it/annunci-friuli-venezia-giulia/vendita/appartamenti/trieste/?o={}'

def get_individual(url):
    req = requests.get(url).text
    soup = BeautifulSoup(req, 'html.parser')
    
    title = soup.find('div', class_='general-info_ad-info___SSdI').find('h1').text if soup.find('div', class_='general-info_ad-info___SSdI') else None
    
    price_tag = soup.find('p', class_='index-module_price__N7M2x AdInfo_ad-info__price__tGg9h index-module_large__SUacX')
    price = price_tag.text.strip() if price_tag else None

    
    main_features_container = soup.find('div', class_='MainData_main-features-container__exoS5')
    if main_features_container:
        main_features = main_features_container.find_all('div', class_='MainData_main-feature__t4TRN')

        my_dictionary = {}

        for feature in main_features:
            img_tag = feature.find('img')
            p_tag = feature.find('p', class_='index-module_sbt-text-atom__ed5J9 index-module_token-caption__TaQWv size-normal index-module_weight-semibold__MWtJJ MainData_label__4od4v')
            if img_tag and p_tag:
                img_alt = img_tag['alt']
                value = p_tag.text.strip()
                my_dictionary[img_alt] = value
    
    
    
    descrizione = soup.find('section', class_='grid_detail-component__7sBtj grid_description__rEv3i').get_text(strip=True)
    
    dettagli_tag = soup.find('div', id='feature-list-section_detail-chip-container__by96k')
    p_tags = dettagli_tag.find_all('p', class_='index-module_sbt-text-atom__ed5J9 index-module_token-caption__TaQWv size-normal index-module_weight-book__WdOfA StaticChip-module_static-chip__va4RV StaticChip-module_medium__OZRaA') if dettagli_tag else []

    # Extract the contents of the p tags and store them in a list
    dettagli = [p.get_text(strip=True) for p in p_tags]
    
    agenzia_tag = soup.find('p', class_='index-module_sbt-text-atom__ed5J9 index-module_token-subheading__SH1NS size-normal index-module_weight-semibold__MWtJJ UserDetails_user-name__sjn9j')
    agenzia = agenzia_tag.text.strip() if agenzia_tag else None

    
   
    
    house_details = {
        'Titolo': title,
        'Link': url,
        'Prezzo': price,
        'Locali': my_dictionary.get('Locali'),
        'MQ': my_dictionary.get('mq'),
        'Bagni': my_dictionary.get('Bagni'),
        'Piano': my_dictionary.get('Piano'),
        'Descrizione': descrizione,
        'Dettagli': dettagli,
        'Agenzia': agenzia
    }
    
    return house_details

def scrape_data():
    url = base_url.format(1)
    req = requests.get(url).text
    soup = BeautifulSoup(req, 'html.parser')
    
    total_pages = (int(soup.find('div', class_='listing-heading ListingHeading_listing-heading__QSQ4k').find('p').text.split()[0]) + 32) // 33


    
    all_urls = []
    for page_number in range(1, total_pages + 1):
        url = base_url.format(page_number)
        req = requests.get(url).text
        soup = BeautifulSoup(req, 'html.parser')
        house_list = soup.find_all('div', class_='items__item item-card item-card--big BigCard-module_card__Exzqv')
        
        for house in house_list:
            house_url = house.find('a')['href']
            all_urls.append(house_url)
    
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(get_individual, url) for url in all_urls]
    
    details_list = [future.result() for future in futures]
    df = pd.DataFrame(details_list)
    
    return df



# Scrape data and print the resulting DataFrame
dataframe = scrape_data()


# Calculate the elapsed time
elapsed_time = time.time() - start_time

# Print the elapsed time
print(f"Elapsed time: {elapsed_time} seconds")


Elapsed time: 179.89383149147034 seconds


In [2]:
dataframe

,Titolo,Link,Prezzo,Locali,MQ,Bagni,Piano,Descrizione,Dettagli,Agenzia
0,COLOGNA - Meraviglioso Trilocale,https://www.subito.it/appartamenti/cologna-mer...,124.000 €,3 Locali,70 mq,1 Bagno,4° Piano,"DescrizioneAppartamento in vendita a Trieste, ...","[Balcone, Ascensore, Arredato]",PARADISI IMMOBILIARE
1,Centro - luminoso quarto piano in stabile d'epoca,https://www.subito.it/appartamenti/centro-lumi...,125.000 €,3 Locali,115 mq,1 Bagno,4° Piano,DescrizioneVia Vasari: LUMINOSO appartamento s...,[Balcone],Opera Immobiliare
2,Nuda proprietà in zona servita,https://www.subito.it/appartamenti/nuda-propri...,54.000 €,2 Locali,59 mq,1 Bagno,Terra,DescrizioneLa NUDA PROPRIETA' è una forma di i...,"[Balcone, Ascensore, Arredato, Aria condizionata]",TECNOCASA - SAN GIOVANNI SRL
3,ZONA CENTRALE - Quadrilocale con Terrazza Abit...,https://www.subito.it/appartamenti/zona-centra...,385.000 €,4 Locali,220 mq,3 Bagni,1° Piano,"DescrizioneIn una zona centrale, pianeggiante ...","[Balcone, Arredato, Aria condizionata]",HomeTrieste San Vito
4,TIBULLO - Elegante appartamento in centro,https://www.subito.it/appartamenti/tibullo-ele...,179.000 €,4 Locali,112 mq,2 Bagni,Rialzato,"DescrizioneNella Centralissima Via Tibullo, a ...","[Balcone, Disponibilità immediata]",PARADISI IMMOBILIARE
...,...,...,...,...,...,...,...,...,...,...
820,Sistiana - borgo san mauro,https://www.subito.it/appartamenti/sistiana-bo...,148.000 €,4 Locali,80 mq,1 Bagno,1° Piano,DescrizioneSISTIANA - BORGO SAN MAURO. \nIN CO...,"[Balcone, Giardino, Ultimo piano, Disponibilit...",CHENI & TUTTA IMMOBILIARE - SOCIETA' IN NOME COLL
821,Vicinanze sentiero Rilke,https://www.subito.it/appartamenti/vicinanze-s...,95.000 €,3 Locali,55 mq,1 Bagno,Rialzato,DescrizioneDUINO - A due passi dal sentiero Ri...,"[Giardino, Disponibilità immediata, Aria condi...",ARMONY IMMOBILIARE di PAHOR ANNAMARIA
822,Duino,https://www.subito.it/appartamenti/duino-tries...,185.000 €,5 Locali,105 mq,2 Bagni,1° Piano,DescrizioneDUINO - In zona super tranquilla ne...,"[Balcone, Disponibilità immediata]",ARMONY IMMOBILIARE di PAHOR ANNAMARIA
823,ADIACENZE VIALE XX SETTEMBRE: APPARTAMENTO MON...,https://www.subito.it/appartamenti/adiacenze-v...,130.000 €,2 Locali,75 mq,1 Bagno,1° Piano,"DescrizioneAdiacenze del Viale XX Settembre, i...",[],SIRIO REAL ESTATE GROUP


In [3]:
826/33

25.03030303030303